In [1]:
import numpy as np
import pandas as pd
import warnings
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import plotly.express as px
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, OneHotEncoder
from sklearn.compose import make_column_transformer, TransformedTargetRegressor
from sklearn.inspection import permutation_importance
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

import warnings

In [2]:
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv('/Users/miguel/HaaS/Module 11_Cap/practical_application_II_starter/data/vehicles.csv')

In [4]:
df.head()

,id,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,VIN,drive,size,type,paint_color,state
0,7222695916,prescott,6000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,az
1,7218891961,fayetteville,11900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ar
2,7221797935,florida keys,21000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fl
3,7222270760,worcester / central MA,1500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ma
4,7210384030,greensboro,4900,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,nc


In [5]:
df.tail()

,id,region,price,year,manufacturer,model,condition,cylinders,fuel,odometer,title_status,transmission,VIN,drive,size,type,paint_color,state
426875,7301591192,wyoming,23590,2019.0,nissan,maxima s sedan 4d,good,6 cylinders,gas,32226.0,clean,other,1N4AA6AV6KC367801,fwd,NaN,sedan,NaN,wy
426876,7301591187,wyoming,30590,2020.0,volvo,s60 t5 momentum sedan 4d,good,NaN,gas,12029.0,clean,other,7JR102FKXLG042696,fwd,NaN,sedan,red,wy
426877,7301591147,wyoming,34990,2020.0,cadillac,xt4 sport suv 4d,good,NaN,diesel,4174.0,clean,other,1GYFZFR46LF088296,NaN,NaN,hatchback,white,wy
426878,7301591140,wyoming,28990,2018.0,lexus,es 350 sedan 4d,good,6 cylinders,gas,30112.0,clean,other,58ABK1GG4JU103853,fwd,NaN,sedan,silver,wy
426879,7301591129,wyoming,30590,2019.0,bmw,4 series 430i gran coupe,good,NaN,gas,22716.0,clean,other,WBA4J1C58KBM14708,rwd,NaN,coupe,NaN,wy


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 426880 entries, 0 to 426879
Data columns (total 18 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   id            426880 non-null  int64  
 1   region        426880 non-null  object 
 2   price         426880 non-null  int64  
 3   year          425675 non-null  float64
 4   manufacturer  409234 non-null  object 
 5   model         421603 non-null  object 
 6   condition     252776 non-null  object 
 7   cylinders     249202 non-null  object 
 8   fuel          423867 non-null  object 
 9   odometer      422480 non-null  float64
 10  title_status  418638 non-null  object 
 11  transmission  424324 non-null  object 
 12  VIN           265838 non-null  object 
 13  drive         296313 non-null  object 
 14  size          120519 non-null  object 
 15  type          334022 non-null  object 
 16  paint_color   296677 non-null  object 
 17  state         426880 non-null  object 
dtypes: f

In [7]:
df.describe()

,id,price,year,odometer
count,4.268800e+05,4.268800e+05,425675.000000,4.224800e+05
mean,7.311487e+09,7.519903e+04,2011.235191,9.804333e+04
std,4.473170e+06,1.218228e+07,9.452120,2.138815e+05
min,7.207408e+09,0.000000e+00,1900.000000,0.000000e+00
25%,7.308143e+09,5.900000e+03,2008.000000,3.770400e+04
50%,7.312621e+09,1.395000e+04,2013.000000,8.554800e+04
75%,7.315254e+09,2.648575e+04,2017.000000,1.335425e+05
max,7.317101e+09,3.736929e+09,2022.000000,1.000000e+07


In [26]:
# Count of values in the range 1 - 50000
count_1_to_50000 = df['odometer'].between(1, 50000).sum()

# Count of values in the range 51000 - 100000
count_51000_to_100000 = df['odometer'].between(51000, 200000).sum()

# Count of values greater than 100000
count_above_100000 = (df['odometer'] > 200000).sum()

count_1_to_50000, count_51000_to_100000, count_above_100000


(133489, 262220, 23170)

In [8]:
# Dropped the specified columns
df = df.drop(['id', 'model','region', 'VIN', 'state', 'title_status'], axis=1)

In [9]:
# List of high-end manufacturers
high_end_manufacturers = ['mercedes-benz', 'bmw', 'audi', 'porsche']

# Filter the DataFrame based on the high-end manufacturers
df1 = df[df['manufacturer'].isin(high_end_manufacturers)]

df1.head()


,price,year,manufacturer,condition,cylinders,fuel,odometer,transmission,drive,size,type,paint_color
91,31990,2018.0,audi,good,NaN,gas,19179.0,other,NaN,NaN,other,NaN
99,29590,2018.0,audi,good,NaN,gas,13035.0,other,fwd,NaN,sedan,NaN
120,6000,2007.0,mercedes-benz,good,6 cylinders,diesel,124000.0,automatic,rwd,NaN,sedan,blue
132,28590,2018.0,audi,good,NaN,gas,31033.0,other,NaN,NaN,SUV,silver
134,29990,2018.0,audi,good,NaN,other,24996.0,automatic,NaN,NaN,sedan,silver


In [10]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
Index: 35473 entries, 91 to 426879
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         35473 non-null  int64  
 1   year          35473 non-null  float64
 2   manufacturer  35473 non-null  object 
 3   condition     21926 non-null  object 
 4   cylinders     18829 non-null  object 
 5   fuel          35378 non-null  object 
 6   odometer      35107 non-null  float64
 7   transmission  35360 non-null  object 
 8   drive         19736 non-null  object 
 9   size          10579 non-null  object 
 10  type          30485 non-null  object 
 11  paint_color   26201 non-null  object 
dtypes: float64(2), int64(1), object(9)
memory usage: 3.5+ MB


In [11]:
# Proportion of null value columns
column_proportions = df.count() / len(df) * 100

print("Proportion of non-null values in each column:")
print(column_proportions)

Proportion of non-null values in each column:
price           100.000000
year             99.717719
manufacturer     95.866286
condition        59.214768
cylinders        58.377530
fuel             99.294181
odometer         98.969265
transmission     99.401237
drive            69.413653
size             28.232524
type             78.247283
paint_color      69.498922
dtype: float64


In [12]:
# Unique Value per Categorical Column
object_columns = df1.select_dtypes(include=['object']).columns

for column in object_columns:
    unique_values = df1[column].unique()
    print(f"Unique values in {column} column: {unique_values}")


Unique values in manufacturer column: ['audi' 'mercedes-benz' 'bmw' 'porsche']
Unique values in condition column: ['good' 'excellent' nan 'fair' 'like new' 'new' 'salvage']
Unique values in cylinders column: [nan '6 cylinders' '4 cylinders' '8 cylinders' '5 cylinders'
 '12 cylinders' '3 cylinders' 'other' '10 cylinders']
Unique values in fuel column: ['gas' 'diesel' 'other' 'electric' 'hybrid' nan]
Unique values in transmission column: ['other' 'automatic' 'manual' nan]
Unique values in drive column: [nan 'fwd' 'rwd' '4wd']
Unique values in size column: [nan 'mid-size' 'full-size' 'sub-compact' 'compact']
Unique values in type column: ['other' 'sedan' 'SUV' 'coupe' 'convertible' nan 'hatchback' 'van' 'wagon'
 'truck' 'bus' 'mini-van' 'pickup' 'offroad']
Unique values in paint_color column: [nan 'blue' 'silver' 'black' 'white' 'red' 'yellow' 'grey' 'custom'
 'brown' 'green' 'orange' 'purple']


In [13]:
# Dropped size column and filtered out N/A odometer values
df2 = df1.drop(columns=['size'])
df2 = df2.dropna(subset=['odometer'])


# Created a list of categorical columns that filled N/A with other
columns_to_replace_nan = ['condition', 'cylinders', 'fuel', 'transmission', 'drive', 'paint_color']
df2[columns_to_replace_nan] = df2[columns_to_replace_nan].fillna('other')

# Replaced uncommon car types with other
valid_types = ['other', 'sedan', 'SUV', 'coupe', 'convertible', 'hatchback']
df2['type'] = df2['type'].apply(lambda x: x if x in valid_types else 'other')

df2.head()

,price,year,manufacturer,condition,cylinders,fuel,odometer,transmission,drive,type,paint_color
91,31990,2018.0,audi,good,other,gas,19179.0,other,other,other,other
99,29590,2018.0,audi,good,other,gas,13035.0,other,fwd,sedan,other
120,6000,2007.0,mercedes-benz,good,6 cylinders,diesel,124000.0,automatic,rwd,sedan,blue
132,28590,2018.0,audi,good,other,gas,31033.0,other,other,SUV,silver
134,29990,2018.0,audi,good,other,other,24996.0,automatic,other,sedan,silver


In [14]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
Index: 35107 entries, 91 to 426879
Data columns (total 11 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   price         35107 non-null  int64  
 1   year          35107 non-null  float64
 2   manufacturer  35107 non-null  object 
 3   condition     35107 non-null  object 
 4   cylinders     35107 non-null  object 
 5   fuel          35107 non-null  object 
 6   odometer      35107 non-null  float64
 7   transmission  35107 non-null  object 
 8   drive         35107 non-null  object 
 9   type          35107 non-null  object 
 10  paint_color   35107 non-null  object 
dtypes: float64(2), int64(1), object(8)
memory usage: 3.2+ MB


In [15]:
# Split the data into features (X) and target variable (y)
X = df2.drop(columns=['price'])
y = df2['price']

In [16]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
# Defined numerical and categorical features for preprocessing
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

In [18]:
# Created a Preprocessing Pipeline, with onehotencoder for the categorical values and scaler for numerical
numeric_transformer = Pipeline(steps=[
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder())
])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])
preprocessor

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('scaler', StandardScaler())]),
                                 Index(['year', 'odometer'], dtype='object')),
                                ('cat',
                                 Pipeline(steps=[('onehot', OneHotEncoder())]),
                                 Index(['manufacturer', 'condition', 'cylinders', 'fuel', 'transmission',
       'drive', 'type', 'paint_color'],
      dtype='object'))])

In [19]:
# Base Pipeline will flow preprocessor features and run through 
pipeline_base = Pipeline([
    ('preprocessor', preprocessor),
    ('poly', PolynomialFeatures()),
    ('target_regressor', TransformedTargetRegressor(regressor=Ridge()))
])

param_grid_base = {
    'poly__degree': [1, 2, 3],
    'target_regressor__regressor__alpha': [0.1, 1, 10]
}
pipeline_base

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('scaler',
                                                                   StandardScaler())]),
                                                  Index(['year', 'odometer'], dtype='object')),
                                                 ('cat',
                                                  Pipeline(steps=[('onehot',
                                                                   OneHotEncoder())]),
                                                  Index(['manufacturer', 'condition', 'cylinders', 'fuel', 'transmission',
       'drive', 'type', 'paint_color'],
      dtype='object'))])),
                ('poly', PolynomialFeatures()),
                ('target_regressor',
                 TransformedTargetRegressor(regressor=Ridge()))])

In [20]:
# Fit the base model
grid_search_base = GridSearchCV(pipeline_base, param_grid_base, cv=5)
grid_search_base.fit(X_train, np.log1p(y_train))


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('preprocessor',
                                        ColumnTransformer(transformers=[('num',
                                                                         Pipeline(steps=[('scaler',
                                                                                          StandardScaler())]),
                                                                         Index(['year', 'odometer'], dtype='object')),
                                                                        ('cat',
                                                                         Pipeline(steps=[('onehot',
                                                                                          OneHotEncoder())]),
                                                                         Index(['manufacturer', 'condition', 'cylinders', 'fuel', 'transmission',
       'drive', 'type', 'paint_color'],
      dtype='object'))])),
                                       ('poly', PolynomialFeatures()),
                                       ('target_regressor',
                                        TransformedTargetRegressor(regressor=Ridge()))]),
             param_grid={'poly__degree': [1, 2, 3],
                         'target_regressor__regressor__alpha': [0.1, 1, 10]})

In [21]:
# Evaluate the base model
test_score_base = grid_search_base.score(X_test, np.log1p(y_test))
print("Base Pipeline Results:")
print(f"Best Hyperparameters: {grid_search_base.best_params_}")
print(f"R-squared on test set: {test_score_base:.4f}")


Base Pipeline Results:
Best Hyperparameters: {'poly__degree': 2, 'target_regressor__regressor__alpha': 10}
R-squared on test set: 0.0828


In [22]:
# Permutation Importance for Base Pipeline
perm_importance_base = permutation_importance(grid_search_base, X_test, np.log1p(y_test), n_repeats=30, random_state=42)
print("Permutation Importance for Base Pipeline:")
print(pd.DataFrame({'Feature': X.columns, 'Importance': perm_importance_base.importances_mean}))

Permutation Importance for Base Pipeline:
        Feature  Importance
0          year   -0.105818
1  manufacturer    0.006275
2     condition    0.237005
3     cylinders    0.179830
4          fuel   -0.039152
5      odometer   -0.088871
6  transmission    0.073172
7         drive    0.145269
8          type    0.120016
9   paint_color    0.063962


In [25]:
# Predict prices using the trained model
y_pred = grid_search_base.predict(X_test)

In [24]:
# Calculate Mean Squared Error
mse = mean_squared_error(y_test, np.expm1(y_pred))
print("Mean Squared Error (MSE):", mse)

Mean Squared Error (MSE): 1.3498218490282093e+78
